<a href="https://colab.research.google.com/github/deterministic-algorithms-lab/indic-asr/blob/main/indic_asr_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/deterministic-algorithms-lab/indic-asr

In [ ]:
%cd indic-asr/

In [ ]:
!pip install -r requirements.txt
!pip install ffmpeg-python

In [ ]:
!sudo apt-get install libenchant1c2a
!pip install pyenchant

## Record Audio to Perform ASR on

In [5]:
#@title Getting Audio from Chrome
"""
To write this piece of code I took inspiration/code from a lot of places.
It was late night, so I'm not sure how much I created or just copied o.O
Here are some of the possible references:
https://blog.addpipe.com/recording-audio-in-the-browser-using-pure-html5-and-minimal-javascript/
https://stackoverflow.com/a/18650249
https://hacks.mozilla.org/2014/06/easy-audio-capture-with-the-mediarecorder-api/
https://air.ghost.io/recording-to-an-audio-file-using-html5-and-js/
https://stackoverflow.com/a/49019356
"""
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import librosa
import io
import ffmpeg

AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");

my_btn.appendChild(t);
//my_p.appendChild(my_btn);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
    mimeType : 'audio/webm;codecs=opus'
    //mimeType : 'audio/webm;codecs=pcm'
  };            
  //recorder = new MediaRecorder(stream, options);
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {            
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data); 
    reader.onloadend = function() {
      base64data = reader.result;
      //console.log("Inside FileReader:" + base64data);
    }
  };
  recorder.start();
  };

recordButton.innerText = "Recording... press to stop";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Saving the recording... pls wait!"
  }
}

// https://stackoverflow.com/a/951057
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
//recordButton.addEventListener("click", toggleRecording);
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  // wait 2000ms for the data to be available...
  // ideally this should use something like await...
  //console.log("Inside data:" + base64data)
  resolve(base64data.toString())

});

}
});
      
</script>
"""

def get_audio():
  display(HTML(AUDIO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])
  
  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, err = process.communicate(input=binary)
  
  riff_chunk_size = len(output) - 8
  # Break up the chunk size into four bytes, held in b.
  q = riff_chunk_size
  b = []
  for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)

  # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
  riff = output[:4] + bytes(b) + output[8:]

  audio, sr = librosa.load(io.BytesIO(riff))

  return audio, sr

In [6]:
audio, sr = get_audio()
audio = librosa.resample(audio, sr, 16000)

## Download and save weights

In [5]:
!rm -r facebook/wav2vec2-base-960h_14/

In [ ]:
import wandb
api = wandb.Api()
run = api.run("interspeech-asr/wav2vec2.0/3bh42rgp/")
run.file("facebook/wav2vec2-base-960h_14/pytorch_model.bin").download()

## Run Model & get Predictions

In [ ]:
%cd src

In [ ]:
from configs import config, get_all_params_dict
from model import get_model
from tokenizer import Wav2Vec2Tok

In [10]:
config.prev_checkpoint = "../facebook/wav2vec2-base-960h_14/"

In [11]:
import torch

In [ ]:
all_params_dict = get_all_params_dict(config)
tokenizer = Wav2Vec2Tok.from_pretrained(config.model)
model = get_model(tokenizer, n_langs=1)
model.load_state_dict(torch.load(config.prev_checkpoint+"pytorch_model.bin", map_location=torch.device("cpu")), strict=False)

In [13]:
speech = tokenizer([audio], return_tensors='pt', padding='longest').input_values
logits = model(speech).logits
predicted_ids = torch.argmax(logits, dim=-1)
predicted_text = tokenizer.batch_decode(predicted_ids)
prediction = tokenizer.revert_transliteration(predicted_text)

In [ ]:
print(prediction)